In [1]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Data preprocessing

dataset = pd.read_csv('Churn_Modelling.csv')
# Exclude row number, customer id, and surname from input since they are unlikely to affect customer churn
X = dataset.iloc[:, 3:-1].values
Y = dataset.iloc[:, -1].values

In [3]:
# Encode categorical variables - geography and gender

# Label encoding for gender
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

# One-hot encoding for geography which is column indexed at 1
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
# Dummy variable trap?

In [4]:
# Split dataset into training set and test set

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [5]:
# Feature scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [6]:
# Initialize ANN

# Create variable to hold ANN
# ANN is of Sequential class
ann = tf.keras.models.Sequential()

# Layers are of Dense class

# Add input layer and first hidden layer
# units correspond to number of neurons in hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Add second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Add output layer
# units = 1 because dependent variable is binary
# activation = softmax for non-binary classification
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [7]:
# Compile ANN

# loss = binary_crossentropy for binary classification
# loss = categorical_crossentropy for non-binary classification
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# Train ANN

ann.fit(X_train, Y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.7350 - accuracy: 0.5470
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5060 - accuracy: 0.8004
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4478 - accuracy: 0.8148
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4199 - accuracy: 0.8245
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3995 - accuracy: 0.8344
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3848 - accuracy: 0.8404
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3748 - accuracy: 0.8451
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3674 - accuracy: 0.8478
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3623 - accuracy: 0.8485
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3580 - accura

In [10]:
# Predict result of single observation

# Use transform() and not fit_transform() on new observations
# Can determine probability threshold beyond which customer churn does not occur
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

array([[0.0338181]], dtype=float32)

In [11]:
# Predict test set results

Y_pred = ann.predict(X_test)
# Convert probability values to binary values
Y_pred = Y_pred > 0.5
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), axis=1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [12]:
# Construct confusion matrix


from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
accuracy_score(Y_test, Y_pred)

[[1529   66]
 [ 203  202]]


0.8655